In [1]:
import json
import pandas as pd
import os

# --- Define the path to our data ---
COMPETITION_NAME = 'fds-pokemon-battles-prediction-2025'
DATA_PATH = os.path.join('../input', COMPETITION_NAME)

train_file_path = os.path.join(DATA_PATH, 'train.jsonl')
test_file_path = os.path.join(DATA_PATH, 'test.jsonl')
train_data = []

# Read the file line by line
print(f"Loading data from '{train_file_path}'...")
try:
    with open(train_file_path, 'r') as f:
        for line in f:
            # json.loads() parses one line (one JSON object) into a Python dictionary
            train_data.append(json.loads(line))

    print(f"Successfully loaded {len(train_data)} battles.")

    # Let's inspect the first battle to see its structure
    print("\n--- Structure of the first train battle: ---")
    if train_data:
        first_battle = train_data[0]
        
        # To keep the output clean, we can create a copy and truncate the timeline
        battle_for_display = first_battle.copy()
        battle_for_display['battle_timeline'] = battle_for_display.get('battle_timeline', [])[:2] # Show first 2 turns
        
        # Use json.dumps for pretty-printing the dictionary
        print(json.dumps(battle_for_display, indent=4))
        if len(first_battle.get('battle_timeline', [])) > 3:
            print("    ...")
            print("    (battle_timeline has been truncated for display)")


except FileNotFoundError:
    print(f"ERROR: Could not find the training file at '{train_file_path}'.")
    print("Please make sure you have added the competition data to this notebook.")

Loading data from '../input/fds-pokemon-battles-prediction-2025/train.jsonl'...
Successfully loaded 10000 battles.

--- Structure of the first train battle: ---
{
    "player_won": true,
    "p1_team_details": [
        {
            "name": "starmie",
            "level": 100,
            "types": [
                "psychic",
                "water"
            ],
            "base_hp": 60,
            "base_atk": 75,
            "base_def": 85,
            "base_spa": 100,
            "base_spd": 100,
            "base_spe": 115
        },
        {
            "name": "exeggutor",
            "level": 100,
            "types": [
                "grass",
                "psychic"
            ],
            "base_hp": 95,
            "base_atk": 95,
            "base_def": 85,
            "base_spa": 125,
            "base_spd": 125,
            "base_spe": 55
        },
        {
            "name": "chansey",
            "level": 100,
            "types": [
                "normal",

In [2]:
from tqdm.notebook import tqdm
import numpy as np

    # enemy_def = moves that deal 1/2x to other pokemons
    # enemy_atk = moves that deal 2x to other pokemons

def types_mult(team: dict) -> dict:
    # no_effect = moves that deal 0x to my pokemons
    # def_weak = moves that deal 2x to my pokemons
    # atk_weak = moves that deal 1/2x to my pokemons
    
    types_dict = {
        "normal": {"no_effect": ["ghost"], "def_weak": ["fighting"], "atk_weak": []}, 
        "fire":   {"no_effect": [], "def_weak": ["water", "ground", "rock"], "atk_weak": ["fire", "grass", "bug"]}
    }
    pass

def static_features(battle: dict) -> dict: 

    features = {}
    stats = ["hp", "spe", "atk", "def", "spd", "spa"]

    # --- Player 1 Team Features ---
    p1_team = battle.get('p1_team_details', [])
    if p1_team:
        # Average stats for p1 team
        for stat in stats:
            features[f'p1_mean_{stat}'] = np.mean([p.get(f'base_{stat}', 0) for p in p1_team])

        

    # --- Player 2 Lead Features ---
    p2_lead = battle.get('p2_lead_details')
    if p2_lead:
        # Stats for lead pokemon p2
        for stat in stats:
            features[f'p2_lead_{stat}'] = p2_lead.get(f'base_{stat}', 0)


    # --- Lead matchup (from first turn) ---
    battle_timeline = battle.get('battle_timeline', [])
    if p1_team and p2_lead and battle_timeline:
        first_turn = battle_timeline[0]
        p1_pokemon_name = first_turn.get('p1_pokemon_state', {}).get('name')

        # Find matching Pokémon in p1_team
        p1_pokemon = next((p for p in p1_team if p.get('name') == p1_pokemon_name), None)

        if p1_pokemon: 
            p1_spe = p1_pokemon.get('base_spe', 0)
            p2_spe = features['p2_lead_spe']
            features['spe_diff'] = p1_spe - p2_spe
        else: 
            features['spe_diff'] = 0.0

    return features


# TODO - Risistemare funzione
def dynamic_features(battle: dict) -> dict:

    features = {}

    p1_hp_loss = 0.0
    p2_hp_loss = 0.0
    prev_p1_hp = None
    prev_p2_hp = None
    
    battle_timeline = battle.get('battle_timeline', [])

    for turn in battle_timeline:
        p1_pokemon_state = turn.get('p1_pokemon_state', {})
        p2_pokemon_state = turn.get('p2_pokemon_state', {})
        
        p1_status = p1_pokemon_state.get('status', {})
        p2_status = p2_pokemon_state.get('status', {})

        p1_hp = p1_pokemon_state.get("hp_pct", 1.0)
        p2_hp = p2_pokemon_state.get("hp_pct", 1.0)

        # HP loss 
        if prev_p1_hp is not None:
            d = p1_hp - prev_p1_hp
            if d < 0:
                p1_hp_loss += -d
        if prev_p2_hp is not None:
            d = p2_hp - prev_p2_hp
            if d < 0:
                p2_hp_loss += -d

        prev_p1_hp = p1_hp
        prev_p2_hp = p2_hp

        # Number of fainted pokemons
        if p1_status == 'fnt': 
            key = "p1_ko_count"
            features[key] = features.get(key, 0) + 1
            
        if p2_status == 'fnt': 
            key = "p2_ko_count"
            features[key] = features.get(key, 0) + 1

        # Number of turns with altered status
        if p1_status not in ['nostatus', 'fnt']:
            key = 'p1_bad_status'
            features[key] = features.get(key, 0) + 1

        if p2_status not in ['nostatus', 'fnt']:
            key = 'p2_bad_status'
            features[key] = features.get(key, 0) + 1

    features["p1_hp_loss_total"] = round(p1_hp_loss * 100, 2)
    features["p2_hp_loss_total"] = round(p2_hp_loss * 100, 2)
            
    return features
    

def create_simple_features(data: list[dict]) -> pd.DataFrame:
    """
    A very basic feature extraction function.
    It only uses the aggregated base stats of the player's team and opponent's lead.
    """
    feature_list = []
    for battle in tqdm(data, desc="Extracting features"):
        features = {}

        features.update(static_features(battle))
        features.update(dynamic_features(battle))

        # We also need the ID and the target variable (if it exists)
        features['battle_id'] = battle.get('battle_id')
        if 'player_won' in battle:
            features['player_won'] = int(battle['player_won'])
            
        feature_list.append(features)
        
    return pd.DataFrame(feature_list).fillna(0)

# Create feature DataFrames for both training and test sets
print("Processing training data...")
train_df = create_simple_features(train_data)

print("\nProcessing test data...")
test_data = []
with open(test_file_path, 'r') as f:
    for line in f:
        test_data.append(json.loads(line))
test_df = create_simple_features(test_data)

print("\nTraining features preview:")
display(train_df.head())

Processing training data...


Extracting features:   0%|          | 0/10000 [00:00<?, ?it/s]


Processing test data...


Extracting features:   0%|          | 0/5000 [00:00<?, ?it/s]


Training features preview:


,p1_mean_hp,p1_mean_spe,p1_mean_atk,p1_mean_def,p1_mean_spd,p1_mean_spa,p2_lead_hp,p2_lead_spe,p2_lead_atk,p2_lead_def,...,p2_lead_spa,spe_diff,p2_bad_status,p1_bad_status,p1_ko_count,p2_ko_count,p1_hp_loss_total,p2_hp_loss_total,battle_id,player_won
0,115.833333,80.000000,72.500000,63.333333,100.000000,100.000000,60,115,75,85,...,100,0,16.0,6.0,1.0,1.0,315.16,351.56,0,1
1,123.333333,61.666667,72.500000,65.833333,90.000000,90.000000,55,120,50,45,...,135,-25,5.0,8.0,3.0,0.0,452.00,230.00,1,1
2,124.166667,65.833333,84.166667,71.666667,90.000000,90.000000,250,50,5,5,...,105,5,14.0,14.0,1.0,0.0,188.00,208.00,2,1
3,121.666667,75.833333,77.500000,65.833333,103.333333,103.333333,75,110,100,95,...,70,0,5.0,17.0,3.0,0.0,395.00,303.00,3,1
4,114.166667,72.500000,75.833333,79.166667,97.500000,97.500000,60,115,75,85,...,100,5,22.0,8.0,1.0,0.0,285.00,387.00,4,1


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define our features (X) and target (y)
features = [col for col in train_df.columns if col not in ['battle_id', 'player_won']]
X_train = train_df[features]
y_train = train_df['player_won']


# split()  method generate indices to split data into training and test set.
for count, (train_index, test_index) in enumerate(kf.split(X_train, y_train)):
    print(f'Fold:{count}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    count += 1

# Define test features
X_test = test_df[features]

# Create pipeline
model = make_pipeline(StandardScaler(), 
                      LogisticRegression(random_state=42, max_iter=1000))

# Cross validation
score = cross_val_score(model, X_train, y_train, cv= kf, scoring="accuracy")
print(f'Scores for each fold are: {score}')
print(f'Average score: {"{:.2f}".format(score.mean())}')

print("Cross validation complete.")

Fold:0, Train set: 8000, Test set:2000
Fold:1, Train set: 8000, Test set:2000
Fold:2, Train set: 8000, Test set:2000
Fold:3, Train set: 8000, Test set:2000
Fold:4, Train set: 8000, Test set:2000
Scores for each fold are: [0.784  0.791  0.786  0.787  0.7875]
Average score: 0.79
Cross validation complete.


In [4]:
# Train the model
print("Training a simple Logistic Regression model...")
model.fit(X_train, y_train)
print("Model training complete.")


# Make predictions on the test data
print("Generating predictions on the test set...")
test_predictions = model.predict(X_test)

# Create the submission DataFrame
submission_df = pd.DataFrame({
    'battle_id': test_df['battle_id'],
    'player_won': test_predictions
})

# Save the DataFrame to a .csv file
submission_df.to_csv('submission.csv', index=False)

print("\n'submission.csv' file created successfully!")
display(submission_df.head())

Training a simple Logistic Regression model...
Model training complete.
Generating predictions on the test set...

'submission.csv' file created successfully!


,battle_id,player_won
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1
